## Environment Setup
Install required libraries 


In [ ]:
# Install essential libraries
!pip install --upgrade gradio


## Interactive UI (Gradio)
Launch a web UI for real-time generation and editing.
Steps:

1. Define UI functions.
2. Build Gradio blocks with tabs for T2I, Editing, and Dataset Browser.

In [ ]:
# Interactive UI (Gradio)
import gradio as gr

_t2i_pipe = {"pipe": None}
_edit_pipe = {"pipe": None}
DATASET_IMAGES = discover_images(DATA_DIRS) if 'DATA_DIRS' in globals() else []
DATASET_CHOICES = [p.name for p in DATASET_IMAGES]

def pick_dataset_image(name):
    if not name: return None
    p = next((p for p in DATASET_IMAGES if p.name==name), None)
    if p is None: return None
    return Image.open(p).convert("RGB")

def ui_t2i(prompt, steps, guidance, width, height, seed):
    if _t2i_pipe["pipe"] is None:
        _t2i_pipe["pipe"] = build_txt2img()
    img = run_t2i(_t2i_pipe["pipe"], prompt, seed, steps, guidance, width, height)
    save_path = Path(OUT_DIR)/"ui_txt2img"/f"t2i_{timestamp()}.png"
    save_path.parent.mkdir(parents=True, exist_ok=True)
    img.save(save_path)
    return img, str(save_path)

def ui_edit(image, instruction, steps, guidance, image_guidance, seed):
    if _edit_pipe["pipe"] is None:
        _edit_pipe["pipe"] = build_edit()
    init = image.convert("RGB")
    img = run_edit(_edit_pipe["pipe"], init, instruction, seed, steps, guidance, image_guidance)
    save_path = Path(OUT_DIR)/"ui_edit"/f"edit_{timestamp()}.png"
    save_path.parent.mkdir(parents=True, exist_ok=True)
    img.save(save_path)
    return img, str(save_path)

with gr.Blocks(title="StepX1Edit - Facade Studio") as demo:
    gr.Markdown("## 🧱 StepX1Edit – Facade Studio (Colab)\nText→Image & Text-Guided Editing")
    with gr.Tab("Text → Image"):
        prompt = gr.Textbox(label="Prompt", placeholder="e.g., modern minimalist glass facade with LED signage")
        with gr.Row():
            steps = gr.Slider(4, 40, value=20, step=1, label="Steps")
            guidance = gr.Slider(0.5, 7.5, value=2.0, step=0.1, label="Guidance")
            width = gr.Slider(512, 1536, value=1024, step=64, label="Width")
            height = gr.Slider(512, 1536, value=1024, step=64, label="Height")
            seed = gr.Number(value=-1, label="Seed (-1=random)")
        btn = gr.Button("Generate")
        out_img = gr.Image(label="Result", interactive=False)
        out_path = gr.Textbox(label="Saved to", interactive=False)
        btn.click(ui_t2i, [prompt, steps, guidance, width, height, seed], [out_img, out_path])

    with gr.Tab("Edit Existing Image"):
        image = gr.Image(type="pil", label="Upload or pick from dataset tab")
        instruction = gr.Textbox(label="Instruction", placeholder="e.g., Convert to Islamic mashrabiya style with geometric patterns")
        with gr.Row():
            e_steps = gr.Slider(4, 40, value=20, step=1, label="Steps")
            e_guid = gr.Slider(0.5, 10.0, value=1.8, step=0.1, label="Guidance")
            e_img_guid = gr.Slider(0.5, 5.0, value=1.5, step=0.1, label="Image Guidance")
            e_seed = gr.Number(value=-1, label="Seed (-1=random)")
        e_btn = gr.Button("Edit")
        e_img = gr.Image(label="Edited", interactive=False)
        e_path = gr.Textbox(label="Saved to", interactive=False)
        e_btn.click(ui_edit, [image, instruction, e_steps, e_guid, e_img_guid, e_seed], [e_img, e_path])

    with gr.Tab("Dataset Browser"):
        ds_dd = gr.Dropdown(choices=DATASET_CHOICES, label="Dataset images (from DATA_DIRS)")
        ds_btn = gr.Button("Load to preview")
        ds_img = gr.Image(label="Preview", interactive=False)
        ds_btn.click(pick_dataset_image, ds_dd, ds_img)

demo.queue().launch(share=True)


## Batch Processing
Automated processing of entire datasets with metadata tracking.

Steps:

1. Save with metadata.
2. Run batch edit or T2I.

In [ ]:
# Batch Processing
def save_with_meta(img, out_path: Path, meta: dict):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    img.save(out_path)
    with open(out_path.with_suffix(".json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

def batch_edit(prompts_file: str, data_dirs: List[str], out_dir: str,
               steps=20, guidance=1.8, image_guidance=1.5):
    df = load_prompts_table(prompts_file).fillna("")
    images = discover_images(data_dirs)
    lookup = {p.name: p for p in images}
    pipe = build_edit()
    out_root = Path(out_dir)/f"edit_{timestamp()}"
    for i, row in df.iterrows():
        img_name = str(row.get("image","")).strip()
        instr = str(row.get("prompt","")).strip()
        if not img_name or not instr: continue
        src = lookup.get(Path(img_name).name, None)
        if src is None and Path(img_name).exists():
            src = Path(img_name)
        if src is None:
            print(f"[WARN] Not found: {img_name}"); continue
        init = Image.open(src).convert("RGB")
        img = run_edit(pipe, init, instr, seed=-1, steps=steps, guidance=guidance, image_guidance=image_guidance)
        out_path = out_root / f"{src.stem}_edited.png"
        meta = {"mode":"edit","instruction":instr,"src":str(src),
                "steps":steps,"guidance":guidance,"image_guidance":image_guidance,"model":EDIT_MODEL}
        save_with_meta(img, out_path, meta)
    print(f"✅ Done → {out_root}")

def batch_t2i(prompts_file: str, out_dir: str, steps=20, guidance=2.0, width=1024, height=1024):
    df = load_prompts_table(prompts_file).fillna("")
    pipe = build_txt2img()
    out_root = Path(out_dir)/f"t2i_{timestamp()}"
    for i, row in df.iterrows():
        prompt = str(row.get("prompt","")).strip()
        if not prompt: continue
        img = run_t2i(pipe, prompt, seed=-1, steps=steps, guidance=guidance, width=width, height=height)
        out_path = out_root / f"t2i_{i:04d}.png"
        meta = {"mode":"txt2img","prompt":prompt,"steps":steps,"guidance":guidance,"model":TXT2IMG_MODEL,"size":[width,height]}
        save_with_meta(img, out_path, meta)
    print(f"✅ Done → {out_root}")
